In [22]:
from keras import models
from keras import layers
from keras import callbacks
import keras_tuner
from keras.datasets import fashion_mnist
import numpy as np

In [11]:
#preparing data
num_classes = 10
num_features = 784
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data, test_data = np.array(train_data, np.float32), np.array(test_data, np.float32)
train_data, test_data = train_data / 255., test_data / 255.

In [18]:
#create_model
def create_model(hp):
    model = models.Sequential([
        layers.Conv2D(hp.Int("filters1", min_value=32, max_value=128, step=32), (3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(hp.Int("filters2", min_value=32, max_value=128, step=32), (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(hp.Int("filters3", min_value=32, max_value=128, step=32), (3, 3), activation="relu"),

        layers.Flatten(),
        layers.Dense(hp.Int("neurons", min_value=16, max_value=128, step=32), activation="relu")
    ])
    if hp.Boolean('dropout'):
        model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation="softmax"))
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

create_model(keras_tuner.HyperParameters())

In [23]:
#tuning
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)          
tuner = keras_tuner.Hyperband(hypermodel=create_model, max_epochs=10, objective="val_accuracy")
tuner.search(train_data, train_labels, validation_data=(test_data, test_labels))

Trial 30 Complete [00h 47m 10s]
val_accuracy: 0.911899983882904

Best val_accuracy So Far: 0.9125999808311462
Total elapsed time: 08h 45m 13s


In [26]:
#building the optimal model
best_hps = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hps)
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data, test_labels), callbacks=[stop_early])

Epoch 1/20
1875/1875 [==============================] - 170s 89ms/step - loss: 0.6110 - accuracy: 0.7745 - val_loss: 0.4081 - val_accuracy: 0.8521
Epoch 2/20
1875/1875 [==============================] - 166s 89ms/step - loss: 0.3628 - accuracy: 0.8688 - val_loss: 0.3407 - val_accuracy: 0.8780
Epoch 3/20
1875/1875 [==============================] - 174s 93ms/step - loss: 0.3084 - accuracy: 0.8881 - val_loss: 0.3214 - val_accuracy: 0.8868
Epoch 4/20
1875/1875 [==============================] - 172s 92ms/step - loss: 0.2747 - accuracy: 0.9000 - val_loss: 0.2941 - val_accuracy: 0.8925
Epoch 5/20
1875/1875 [==============================] - 183s 98ms/step - loss: 0.2500 - accuracy: 0.9092 - val_loss: 0.2788 - val_accuracy: 0.9038
Epoch 6/20
1875/1875 [==============================] - 174s 93ms/step - loss: 0.2253 - accuracy: 0.9185 - val_loss: 0.2835 - val_accuracy: 0.9001
Epoch 7/20
1875/1875 [==============================] - 170s 90ms/step - loss: 0.2074 - accuracy: 0.9235 - val_loss: 0

In [27]:
#getting accuracy
model.evaluate(test_data, test_labels)
#В задані написано "порівняйте точність з мережою з останнього завдання", але в останньому завданні мені довелось використати СNN
#бо повнозв'язна мережа ніяк не могла досягти необхідної точності.
#Що цікаво, ця мережа дала меншу точність ніж в попередньму завданні. Думаю це тому що Hyberband відмовився використовувати dropout
#Але вже буде так. Я не хочу чекати ще 8 годин поки підберуться параметри.

  6/313 [..............................] - ETA: 8s - loss: 0.3673 - accuracy: 0.9062

313/313 [==============================] - 8s 26ms/step - loss: 0.3086 - accuracy: 0.9086


[0.30856600403785706, 0.9085999727249146]